## 1. 라이브러리 및 데이터 불러오기

In [1]:
import os
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup as bs
from tqdm.notebook import tqdm
from string import punctuation
import warnings
warnings.filterwarnings('ignore')

In [5]:
jobplanet = pd.read_excel('C:/Users/hanah/Desktop/workspace/Project/ESG/preprocessing/jobplanet/jobplanet.xlsx')

In [7]:
path = 'C:/Users/hanah/Desktop/workspace/Project/ESG/modelling/polarity_scoring/통계청/'
file_list = os.listdir(path)

In [8]:
df = pd.DataFrame()
for file in file_list:
    data = pd.read_excel(path + file)
    df = pd.concat([df, data])
    
df.reset_index(drop = True, inplace = True)

## 2. 네이버 뉴스 학습데이터 라벨링 조정

In [9]:
df['라벨'] = df.iloc[:, 3:9].mean(axis=1)
df['라벨'][df['라벨'] <= 2.5] = 0
df['라벨'][df['라벨'] > 2.5] = 1
df['라벨'].value_counts()

1.0    4459
0.0    3159
Name: 라벨, dtype: int64

In [10]:
df = df[['제목', 'URL', '라벨']]
df

,제목,URL,라벨
0,"[신년사]손경식 CJ회장_ ""해외사업 공격적 확장_ 성장 가속화""",http://news.naver.com/main/read.nhn?mode=LSD&m...,1.0
1,[취업과 창업]신입사원 연차 생기고 공공기관 채용 역대 최대,http://news.naver.com/main/read.nhn?mode=LSD&m...,1.0
2,2018년 채용연계형 인턴채용 문 열렸다,http://news.naver.com/main/read.nhn?mode=LSD&m...,1.0
3,무술년 채용연계형 인턴채용 스타트,http://news.naver.com/main/read.nhn?mode=LSD&m...,1.0
4,그냥 쉬었다 청년_ 30만명 역대 최고치,http://news.naver.com/main/read.nhn?mode=LSD&m...,0.0
...,...,...,...
7613,코로나 충격에 따른 근로 시간 감소_ 과거 위기의 5배,https://news.naver.com/main/read.nhn?mode=LSD&...,0.0
7614,7월 사업체종사자 13만8천명 감소 코로나19 여파에 5개월째 감소,https://news.naver.com/main/read.nhn?mode=LSD&...,0.0
7615,한치 앞도 안보인다..채용문 꽁꽁 닫는 카드사,https://news.naver.com/main/read.nhn?mode=LSD&...,0.0
7616,토스_ 고객 상담직 신입 및 경력 30명 공개 채용,https://news.naver.com/main/read.nhn?mode=LSD&...,1.0


## 3. 기사 본문 크롤링 후 전처리

In [52]:
temp = pd.read_csv('news.csv', encoding='cp949')
temp = temp.drop(['categories', 'dates', 'titles'], axis=1)
temp.columns = ['기사', 'URL']
temp

,기사,URL
0,청년 실업률이 역대 최고 수준을 기록한 가운데 경제활동도 없이 쉬는 청년이 작년에 ...,http://news.naver.com/main/read.nhn?mode=LSD&m...
1,기업은행 노사가 무기계약직 3300명에 대한 정규직 전환을 추진한다.2일 기업은행 ...,http://news.naver.com/main/read.nhn?mode=LSD&m...
2,대학가에 취업 훈풍이 불고 있다.정부가 공공부문 채용 규모를 확대하고 대기업이 신규...,http://news.naver.com/main/read.nhn?mode=LSD&m...
3,올해 정규직 고용과 연관된 채용연계형 인턴사원 모집이 시작됐다.9일 취업 포털 업체...,http://news.naver.com/main/read.nhn?mode=LSD&m...
4,[한겨레]최저임금 인상이 청년고용에 미치는 부정적 영향은 없거나 통계적으로 무의미할...,http://news.naver.com/main/read.nhn?mode=LSD&m...
...,...,...
7491,(서울=연합뉴스) 성서호 기자 = 신종 코로나바이러스 감염증(코로나19) 확산에 따...,https://news.naver.com/main/read.nhn?mode=LSD&...
7492,[머니투데이 김세관 기자]간편송금 플랫폼 '토스' 운영사 비바리퍼블리카는 31일 고...,https://news.naver.com/main/read.nhn?mode=LSD&...
7493,지난달 국내 사업체 종사자 수가 1년 전보다 13만8천명 줄어들었다. 코로나19 여...,https://news.naver.com/main/read.nhn?mode=LSD&...
7494,[서울=뉴시스] 김진아 기자 = 신종 코로나바이러스 감염증(코로나19)으로 인한 고...,https://news.naver.com/main/read.nhn?mode=LSD&...


In [53]:
temp = temp.dropna(axis=0)
temp = temp.loc[temp['기사'].str.len() > 100]
temp.drop_duplicates(subset=['기사'], inplace=True)

In [51]:
def clean_str(text):
    text = re.sub('[a-zA-Z-一-龥]','', text) # 영어, 숫자 제거
    text = re.sub('([ㄱ-ㅎㅏ-ㅣ]+)','', text)  # 자음, 모음 제거
    text = re.sub('<[^>]*>', '', text) # HTML 태그 제거 
    text = re.sub('(\【(.*?)\】)|(\[(.*?)\])|(\((.*?)\))', '', text) # 괄호안 글자 제거
    text = re.sub('[^\w\s^%^~^!^?^.]', ' ', text)  # 특수기호 제거
    pattern = '\.([^\.]*(?:기자|뉴스|특파원|교수|작가|대표|논설|주필|에세이|화백|사설|소장|단장|큐레이터|저작권|평론가|©|©|ⓒ|\@|\/|=|:앞쪽_화살표:|무단|전재|재배포|금지|\[|\]|\(\))[^\.]*)'
    text = re.sub(pattern, '', text) # 바이라인 제거    
    text = re.sub("\.+\.", '.', text)  # 다수의 점 (ex : ...) 점 한개로 대체
    text = re.sub(' +', ' ', text)   # 다수의 공백 축소
    text = re.sub('([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)', '', text) # 이메일 제거
    text = re.sub('(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', '', text) # url 제거
    text = re.sub('([가-힣]{2,3}뉴스|[가-힣]{2,3} 기자)', '', text) # 매체, 기자 이름 제거
    text = text.replace('Copyrights', '').replace('무단전재 및 재배포 금지', '').replace('무단복제','').replace('무단배포','')
    text = text.replace('재배포 금지','').replace('무단 전재','')
    return text 

In [55]:
temp['기사'] = temp['기사'].map(clean_str)
df['제목'] = df['제목'].map(clean_str)

In [56]:
news_train = pd.merge(df, temp, on='URL', how='left')
news_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7618 entries, 0 to 7617
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   제목      7618 non-null   object 
 1   URL     7606 non-null   object 
 2   라벨      7618 non-null   float64
 3   기사      7243 non-null   object 
dtypes: float64(1), object(3)
memory usage: 297.6+ KB


In [68]:
for idx in range(len(news_train)):
    if type(news_train.기사[idx]) == float: 
        news_train.loc[idx, '기사'] = news_train.제목[idx]

In [74]:
news_train.to_excel('news_train.xlsx', index=False)

## 4. 잡플래닛 데이터 라벨링 조정

In [8]:
jobplanet['label'] = 0
jobplanet['label'].loc[jobplanet['TotScore'] >= 50] = 1
jobplanet['label'].loc[jobplanet['TotScore'] < 50] = 0
jobplanet['label'].value_counts()

0    29779
1    27670
Name: label, dtype: int64

In [13]:
jobplanet.to_excel('jobplanet_train.xlsx', index=False)

## 5. 인풋데이터 생성

In [49]:
train_data = pd.concat([news, jobplanet, nsmc])

In [63]:
train_data['doc'] = train_data['doc'].map(clean_str)

In [64]:
train_data.to_excel('train_data.xlsx', index=False)